In [1]:
import numpy as np
import pandas as pd
import re, os

In [2]:
notes = ['C', 'Db', 'D', 'Eb', 'E', 'F', 'Gb', 'G', 'Ab', 'A', 'Bb', 'B']
note_to_num = {}
for num, note in enumerate(notes):
    note_to_num[note] = num
num_to_note = dict([[v,k] for k,v in note_to_num.items()])
same_note = {'A#':'Bb', 'C#':'Db', 'D#':'Eb', 'F#': 'Gb', 'G#':'Ab'}

def split_note(note):
    assert re.fullmatch('[A-G](#|b)?[0-7]', note) is not None, 'Note not formatted correctly: %s'%note
    return note[:-1], int(note[-1])

def shift_note(note, amount):
    # note taken in as string, amount is any integer
    # probably not needed until actually generating stuff
    note, octave = split_note(note)
    if note in same_note:
        note = same_note[note]
    new_num = note_to_num[note] + amount
    if new_num > 11:
        octave += 1
    elif new_num < 0:
        octave -= 1
    return num_to_note[(new_num) % 12] + str(octave)

def note_dist(note1, note2):
    # positive if note2 is above note1, 0 if same
    note1, octave1 = split_note(note1)
    note2, octave2 = split_note(note2)
    if note1 in same_note:
        note1 = same_note[note]
    if note2 in same_note:
        note2 = same_note[note]
    tot = (octave2 - octave1) * 12
    tot += note_to_num[note2] - note_to_num[note1]
    return tot

### Finding Slope Bounds

In [3]:
def find_slope_bounds(lst):
    max_jump, min_jump = 0, 0
    for i in range(len(lst) - 1):
        max_jump = max(max_jump, note_dist(lst[i], lst[i+1]))
        min_jump = min(min_jump, note_dist(lst[i], lst[i+1]))
    return str(min_jump) + ' ' + str(max_jump)

### Creating the S-Expressions Table

In [4]:
def categorize_note(note, chord, last_chord):
    return np.random.choice(['C', 'H', 'R'])

In [6]:
def create_s_exp(notes):
    # notes is list of tupes of (note_string, duration)
    s = ''
    notes_only = []
    for note, duration, chord, last_chord in notes:
        s += categorize_note(note, chord, last_chord) + str(duration) + ' '
        notes_only.append(note)
    return find_slope_bounds(notes_only) + ' ' + s

In [9]:
s_exp = pd.DataFrame(columns=['id', 'exp', 'song_id', 'song_index'])
s_exp = s_exp.set_index(['id'])
directory = 'raw_solos/'
song_num = 0
for filename in os.listdir(directory):
    if filename.endswith('.csv'):
        measure = 0
        last_chord = None
        curr_s_exp = []
        song = pd.read_csv(directory + filename)
        for i in range(len(song)):
            curr_note = song.iloc[i]
            if measure != int(curr_note['start_time']):
                s = create_s_exp(curr_s_exp)
                row = {'exp': s, 'song_id': song_num, 'song_index': measure}
                s_exp = s_exp.append(row, ignore_index=True)
                curr_s_exp = []
                measure = int(curr_note['start_time'])
            curr_s_exp.append((curr_note['note_name'], curr_note['duration'], curr_note['chord'], last_chord))
            last_chord = curr_note['chord']
        s = create_s_exp(curr_s_exp)
        row = {'exp': s, 'song_id': song_num, 'song_index': measure}
        s_exp = s_exp.append(row, ignore_index=True)
        song_num += 1
song_num

2

In [10]:
s_exp

,exp,song_id,song_index
0,0 2 C0.25 C0.5,0,0
1,-5 0 C0.2 R0.5,0,1
2,0 6 R0.05 C0.35,0,2
3,0 0 H0.4 H0.1,1,0
4,0 0 R0.6 C0.34,1,1
5,0 6 C0.23 R0.53,1,2
